# Project 3 - Data collection
Andy Tan

In [180]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import random
import re

## SQL database creation
Main dataset was obtained through sports-reference.com's API (https://sportsreference.readthedocs.io/en/stable/ncaab.html). Data was scraped for yearly and career stats for all division 1 NCAA basketball players from 2010 - 2019. This was subsequently uploaded in SQL on AWS.

In [7]:
#Database created with postgresql on AWS. Implented code is copied below.
'''create table player_stats (
               index int primary key,
               year varchar(20),
               assist_percentage float,
               assists float,
               block_percentage float,
               blocks float,
               box_plus_minus float,
               conference varchar(20),
               defensive_box_plus_minus float,
               defensive_rebound_percentage float,
               defensive_rebounds float,
               defensive_win_shares float,
               effective_field_goal_percentage float,
               field_goal_attempts float,
               field_goal_percentage float,
               field_goals float,
               free_throw_attempt_rate float,
               free_throw_attempts float,
               free_throw_percentage float,
               free_throws float,
               games_played float,
               games_started float,
               height varchar(20), 
               minutes_played float,
               offensive_box_plus_minus float,
               offensive_rebound_percentage float,
               offensive_rebounds float,
               offensive_win_shares float,
               personal_fouls float,
               player_efficiency_rating float,
               player_id varchar(50),
               points float,
               points_produced float,
               position varchar(20), 
               steal_percentage float,
               steals float,
               team_abbreviation varchar(50), 
               three_point_attempt_rate float,
               three_point_attempts float,
               three_point_percentage float,
               three_pointers float,
               total_rebound_percentage float,
               total_rebounds float,
               true_shooting_percentage float,
               turnover_percentage float,
               turnovers float,
               two_point_attempts float,
               two_point_percentage float,
               two_pointers float,
               usage_percentage float,
               weight float,
               win_shares float,
               win_shares_per_40_minutes float);'''

'''\copy player_stats from 'metis_projects/andy_project_03/data/players_stats_2010-2020.csv' 
                delimiter ',' csv header;'''

## Helper functions

In [174]:
def get_player_id(player_str):
    '''
    Function to generate player ID from player name
    
    args:
        player_str - player name string
    output:
        player_id - player id string
    '''
    
    player_id = player_str.strip().replace("'",'').replace('.','').replace('_',' ').lower().replace(' ','-')
    player_id = player_id.replace('-jr.','').replace('-jr','').replace('-iii','').replace('-ii','')
    
    return player_id


def get_college_id(college_str):
    '''
    Function to generate college ID from player name
    
    args:
        college_str - player name string
    output:
        college_id - player id string
    '''

    college_id = college_str.strip().lower()
    
    if college_id.startswith('university of'):
        college_id = college_id[14:]
    if college_id.endswith('university'):
        college_id = college_id[:-11]
    if college_id.endswith('st.'):
        college_id = college_id.replace('st.','state')
    if college_id == 'california, los angeles':
        college_id = 'ucla'
    if college_id == 'texas at austin':
        college_id = 'texas'
    if ' at ' in college_id:
        college_id = college_id.replace(' at ',' ')        
    if college_id == 'georgia institute of technology':
        college_id = 'georgia tech'
    if college_id == "st. john's":
        college_id = 'st johns ny'
    if college_id == "nevada, reno":
        college_id = 'nevada'
    if college_id == 'college of william & mary':
        college_id = 'william mary'
    if college_id == 'massachusetts amherst':
        college_id = 'massachusetts'
    if college_id == 'illinois urbana-champaign':
        college_id = 'illinois'
    if 'california state university' in college_id:
        college_id = college_id[29:] + ' state'
    if 'virginia polytechnic' in college_id:
        college_id = 'virginia tech'
    if college_id == 'providence college':
        college_id = 'providence'
    if college_id == 'pennsylvania state':
        college_id = 'penn state'
    if college_id == 'wisconsin-green bay':
        college_id = 'green bay'
    if college_id == 'usc':
        college_id = 'southern california'
    if college_id == 'lsu':
        college_id = 'louisiana state'
    if college_id == 'unlv':
        college_id = 'nevada las vegas'
    if 'miami' in college_id:
        college_id = 'miami fl'
    if 'grambling' in college_id:
        college_id = 'grambling'
    if college_id == 'utep':
        college_id = 'texas el paso'
    
    college_id = college_id.replace('technological','tech')
    
    college_id = re.sub(r'[,&.\']+','',college_id).replace(' ','-')
    
    return college_id


def draft_round(pick):
    '''
    Function to convert pick # in NBA draft to round
    
    args:
        pick: spot drafted 
        
    return:
        round: round drafter
    '''
    if pick <= 30:
        round = 1
    else:
        round = 2
   
    return round

def ucstatus_round(status):
    if '1st/' in status:
        round = 1
    elif '2nd/' in status:
        round = 2
    else:
        round = 0
        
    return round

def to_inches(height_str):
    '''
    Convert height string to numerical inches
    
    args:
        height_str: string with height information
        
    return:
        integer of height in inches
    '''
    try:
        height = height_str.strip("''").split("' ")
    
        return float(height[0])*12 + float(height[1])
    
    except:
        return height_str

## Generating prospects database

In [181]:
#NBA draft results
df_draft = pd.read_csv('data/nba_draft_2010-2019.csv', index_col=[0], header=0, names=['player','college','draft_year','pick'])
df_draft.college.value_counts().to_csv('data/colleges1.csv')

In [182]:
df_draft = df_draft.dropna()
df_draft['player_id'] = df_draft['player'].apply(lambda x: get_player_id(x))
df_draft['college_id'] = df_draft['college'].apply(lambda x: get_college_id(x))
df_draft['round'] = df_draft['pick'].apply(lambda x: draft_round(x))
df_draft

,player,college,draft_year,pick,player_id,college_id,round
0,John Wall,University of Kentucky,2010,1,john-wall,kentucky,1
1,Evan Turner,Ohio State University,2010,2,evan-turner,ohio-state,1
2,Derrick Favors,Georgia Institute of Technology,2010,3,derrick-favors,georgia-tech,1
3,Wesley Johnson,Syracuse University,2010,4,wesley-johnson,syracuse,1
4,DeMarcus Cousins,University of Kentucky,2010,5,demarcus-cousins,kentucky,1
...,...,...,...,...,...,...,...
594,Kyle Guy,Virginia,2019,55,kyle-guy,virginia,2
595,Jaylen Hands,UCLA,2019,56,jaylen-hands,ucla,2
596,Jordan Bone,Tennessee,2019,57,jordan-bone,tennessee,2
597,Miye Oni,Yale,2019,58,miye-oni,yale,2


In [372]:
#Scraping underclassmen draft declarations from thedraftreview.com

In [65]:
def get_underclass(url_year):
    bigboard = []
    
    url = "https://www.thedraftreview.com/index.php?option=com_content&view=article&id=" + url_year + "-underclassmen&catid=136&Itemid=384"
    
    print(f'Early Entry Underclassmen: {url_year}')
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    table = soup.find_all('table')
    rows = [row for row in table[-1].find_all('tr')]
    for row in rows[1:]:
        items = row.find_all('td')
        bigboard.append([i.text for i in items])
        time.sleep(.5+random.random())

    return bigboard

In [66]:
years = ['6065:2010','6444:2011','6882:2012','7265:2013','7816:2014',
        '8205:2015','8768:2016','10086:2017','11322:2018','12232:2019']

In [67]:
dfs = []

for year in years:
    uc_list = get_underclass(year)
    uc_df = pd.DataFrame(uc_list[1:], columns=uc_list[0])
    uc_df['draft_year'] = int(year[-4:])
    dfs.append(uc_df)

df = pd.concat(dfs)

Early Entry Underclassmen: 6065:2010
Early Entry Underclassmen: 6444:2011
Early Entry Underclassmen: 6882:2012
Early Entry Underclassmen: 7265:2013
Early Entry Underclassmen: 7816:2014
Early Entry Underclassmen: 8205:2015
Early Entry Underclassmen: 8768:2016
Early Entry Underclassmen: 10086:2017
Early Entry Underclassmen: 11322:2018
Early Entry Underclassmen: 12232:2019


In [98]:
df2 = df.copy()

In [99]:
df2 = df2[df2.Status != 'Withdrew']
df2 = df2.replace('JR..','JR.')
df2 = df2.loc[df2.Class.isin(['JR.','SO.','FR.'])]
df2 = df2.reset_index(drop=True)

In [101]:
df2.to_csv('data/underclass_nba_draft.csv')

In [ ]:
#cleaning undergrad prospects df

In [183]:
df_uc = pd.read_csv('data/underclass_nba_draft2.csv', index_col=[0], header=0, names=['class','college','player','status','draft_year'])
#df_uc.college.value_counts().to_csv('data/colleges2.csv')

In [184]:
df_uc = df_uc.dropna()
df_uc['player_id'] = df_uc['player'].apply(lambda x: get_player_id(x))
df_uc['college_id'] = df_uc['college'].apply(lambda x: get_college_id(x))
df_uc['round'] = df_uc['status'].apply(lambda x: ucstatus_round(x))
df_uc['class'] = df_uc['class'].str.lower()
df_uc

,class,college,player,status,draft_year,player_id,college_id,round
0,so.,Florida St.,Solomon Alabi,Dallas 2nd/50th,2010,solomon-alabi,florida-state,2
1,jr.,Kansas,Cole Aldrich,New Orleans 1st/11th,2010,cole-aldrich,kansas,1
2,so.,Wake Forest,Al-Farouq Aminu,LA Clippers 1st/8th,2010,al-farouq-aminu,wake-forest,1
3,jr.,Oklahoma St.,James Anderson,San Antonio 1st/20th,2010,james-anderson,oklahoma-state,1
4,jr.,Nevada,Luke Babbitt,Minnesota 1st/16th,2010,luke-babbitt,nevada,1
...,...,...,...,...,...,...,...,...
548,so.,Iowa State,Lindell Wigginton,Undrafted,2019,lindell-wigginton,iowa-state,0
549,so.,UCLA,Kris Wilkes,Undrafted,2019,kris-wilkes,ucla,0
550,jr.,Tennessee,Grant Williams,Boston 1st/22nd,2019,grant-williams,tennessee,1
551,fr.,Duke,Zion Williamson,New Orleans 1st/1st,2019,zion-williamson,duke,1


In [222]:
df_uc.to_csv('data/uc_draft.csv')

In [ ]:
#Scraping pre-draft Big Board rankings from nbadraft.net

In [112]:
def get_bb100(year):
    bigboard = []
    
    url = "https://www.nbadraft.net/ranking/bigboard/?year-ranking=" + str(year)
    
    print(f'Big Board 100: {year}')
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    table = soup.find('table')
    rows = [row for row in table.find_all('tr')]
    bigboard.append([i.text for i in rows[0].find_all('th')])
    for row in rows[1:]:
        items = row.find_all('td')
        bigboard.append([i.getText(separator=' ') for i in items])
        time.sleep(.5+random.random())

    return bigboard

In [121]:
dfs = []

for year in range(2010,2020):
    bb_list = get_bb100(year)
    #print(bb_list)
    bb_df = pd.DataFrame(bb_list[1:], columns=bb_list[0])
    bb_df['draft_year'] = year
    dfs.append(bb_df)

df = pd.concat(dfs)

Big Board 100: 2010
Big Board 100: 2011
Big Board 100: 2012
Big Board 100: 2013
Big Board 100: 2014
Big Board 100: 2015
Big Board 100: 2016
Big Board 100: 2017
Big Board 100: 2018
Big Board 100: 2019


In [181]:
df3 = df.copy()

In [182]:
df3 = df3.rename(columns={'':'bb_rank'})
df3 = df3.iloc[:, np.r_[0, 2:9]]
df3 = df3.loc[df3.Class.isin(['Fr.','So.','Jr.','Sr.'])]
df3 = df3.reset_index(drop=True)

In [ ]:
df3.to_csv('data/nba_predraft_bb.csv')

In [ ]:
#cleaning draft big board df

In [185]:
df_bb = pd.read_csv('data/nba_predraft_bb2.csv', index_col=[0], header=0, names=['bb_rank','player','height','weight','pos','college','class','draft_year'])

In [186]:
df_bb = df_bb.dropna()
df_bb['player_id'] = df_bb['player'].apply(lambda x: get_player_id(x))
df_bb['college_id'] = df_bb['college'].apply(lambda x: get_college_id(x))
df_bb['class'] = df_bb['class'].str.lower()
df_bb['pos'] = df_bb['pos'].str.lower()
df_bb

,bb_rank,player,height,weight,pos,college,class,draft_year,player_id,college_id
0,1,John Wall,6-4,196,pg,Kentucky,fr.,2010,john-wall,kentucky
1,2,Evan Turner,6-7,214,sg,Ohio St.,jr.,2010,evan-turner,ohio-state
2,3,Derrick Favors,6-10,245,pf,Georgia Tech,fr.,2010,derrick-favors,georgia-tech
3,4,Wesley Johnson,6-7,206,sf,Syracuse,jr.,2010,wesley-johnson,syracuse
4,5,Greg Monroe,6-11,247,pf/c,Georgetown,so.,2010,greg-monroe,georgetown
...,...,...,...,...,...,...,...,...,...,...
855,96,CJ Massinburg,6-3,195,sg,Buffalo,sr.,2019,cj-massinburg,buffalo
856,97,Cody Martin,6-6,190,sg,Nevada,sr.,2019,cody-martin,nevada
857,98,Jarrey Foster,6-6,220,sf,SMU,sr.,2019,jarrey-foster,smu
858,99,Barry Brown,6-3,195,pg/sg,Kansas State,sr.,2019,barry-brown,kansas-state


In [ ]:
#NBA draft combine data

In [195]:
dfs = []

for year in range(2010,2020):
    anthro_df = pd.read_csv(f'data/combine_anthro_{year}.csv')
    strength_df = pd.read_csv(f'data/combine_strength_{year}.csv')
    combo = pd.merge(anthro_df, strength_df, on=['PLAYER'], how='outer')
    combo['draft_year'] = year
    dfs.append(combo)

df_combine = pd.concat(dfs)

In [196]:
df_combine

,PLAYER,POS_x,BODY FAT %,HAND LENGTH (INCHES),HAND WIDTH (INCHES),HEIGHT W/O SHOES,HEIGHT W/ SHOES,STANDING REACH,WEIGHT (LBS),WINGSPAN,POS_y,LANE AGILITY TIME \n(SECONDS),SHUTTLE RUN \n(SECONDS),THREE QUARTER SPRINT \n(SECONDS),STANDING VERTICAL LEAP \n(INCHES),MAX VERTICAL LEAP \n(INCHES),MAX BENCH PRESS \n(REPETITIONS),draft_year
0,Solomon Alabi,C,5.00%,9.25,9.25,6' 11.5'',7' 0.75'',9' 5'',237.4,7' 5'',C,13.2,-,3.68,22.5,26,10,2010
1,Cole Aldrich,C,8.50%,9,9.25,6' 9'',6' 11.25'',9' 3.5'',236,7' 4.75'',C,11.48,-,3.35,23,28,10,2010
2,Al-Farouq Aminu,SF,8.20%,9.5,9.5,6' 7.25'',6' 8.5'',9' 0.5'',216,7' 3.25'',SF,11.29,-,3.3,27,33.5,13,2010
3,James Anderson,SF,7.90%,8.5,7.25,6' 4.75'',6' 5.75'',8' 8'',207.6,6' 8.5'',SF,11.86,-,3.19,30,35.5,14,2010
4,Luke Babbitt,SF-PF,5.60%,8.5,10,6' 7.5'',6' 8.75'',8' 8.5'',217.8,6' 11.25'',SF-PF,10.98,-,3.4,29.5,37.5,15,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Coby White,PG,4.30%,7.75,9,6' 3.5'',6' 4.75'',8' 1.5'',191.4,6' 5'',PG,-,-,-,-,-,-,2019
73,Kris Wilkes,SF,4.90%,8.5,9.5,6' 6.25'',6' 7.75'',8' 7'',208.8,6' 10.75'',SF,10.39,3.09,3.34,27.5,35,4,2019
74,Grant Williams,PF,5.40%,9,10.5,6' 5.75'',6' 7.5'',8' 8.5'',240.2,6' 9.75'',PF,10.83,3.27,3.33,26,31.5,20,2019
75,Zion Williamson,PF,-%,-,-,NaN,NaN,NaN,-,NaN,PF,-,-,-,-,-,-,2019


In [197]:
df_combine = df_combine.loc[:,['PLAYER','HEIGHT W/ SHOES', 'STANDING REACH', 'WINGSPAN', 'STANDING VERTICAL LEAP \n(INCHES)','MAX VERTICAL LEAP \n(INCHES)','draft_year']]
df_combine.columns = ['player_id','ht_shoes','std_reach','wingspan','std_vert_leap','max_vert_leap','draft_year']
df_combine['player_id'] = df_combine['player_id'].apply(lambda x: get_player_id(x))
df_combine['wingspan'] = df_combine['wingspan'].apply(lambda x: to_inches(x))
df_combine['ht_shoes'] = df_combine['ht_shoes'].apply(lambda x: to_inches(x))
df_combine['std_reach'] = df_combine['std_reach'].apply(lambda x: to_inches(x))
df_combine[['std_vert_leap','max_vert_leap']] = df_combine[['std_vert_leap','max_vert_leap']].apply(pd.to_numeric, errors='coerce')

In [198]:
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 76
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_id      797 non-null    object 
 1   ht_shoes       601 non-null    float64
 2   std_reach      601 non-null    float64
 3   wingspan       602 non-null    float64
 4   std_vert_leap  527 non-null    float64
 5   max_vert_leap  528 non-null    float64
 6   draft_year     797 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 49.8+ KB


## Merging dataframes

In [212]:
df_draft_uc = pd.merge(df_draft[['draft_year','round','player_id','college_id']], df_uc[['draft_year','round','player_id','college_id','class']], on=['player_id','draft_year','round'], how='outer')
df_draft_uc_bb = pd.merge(df_draft_uc, df_bb[['bb_rank','player_id','pos','college_id','class','draft_year']], on=['player_id','draft_year'], how='outer')


In [213]:
df_draft_uc_bb

,draft_year,round,player_id,college_id_x,college_id_y,class_x,bb_rank,pos,college_id,class_y
0,2010,1.0,john-wall,kentucky,kentucky,fr.,1.0,pg,kentucky,fr.
1,2010,1.0,evan-turner,ohio-state,ohio-state,jr.,2.0,sg,ohio-state,jr.
2,2010,1.0,derrick-favors,georgia-tech,georgia-tech,fr.,3.0,pf,georgia-tech,fr.
3,2010,1.0,wesley-johnson,syracuse,syracuse,jr.,4.0,sf,syracuse,jr.
4,2010,1.0,demarcus-cousins,kentucky,kentucky,fr.,7.0,pf/c,kentucky,fr.
...,...,...,...,...,...,...,...,...,...,...
986,2019,NaN,donta-hall,NaN,NaN,NaN,93.0,c,alabama,sr.
987,2019,NaN,bennie-boatwright,NaN,NaN,NaN,95.0,pf,southern-california,sr.
988,2019,NaN,cj-massinburg,NaN,NaN,NaN,96.0,sg,buffalo,sr.
989,2019,NaN,jarrey-foster,NaN,NaN,NaN,98.0,sf,smu,sr.


In [214]:
df_all = pd.merge(df_draft_uc_bb, df_combine, on=['player_id','draft_year'], how='left')

In [215]:
df_all

,draft_year,round,player_id,college_id_x,college_id_y,class_x,bb_rank,pos,college_id,class_y,ht_shoes,std_reach,wingspan,std_vert_leap,max_vert_leap
0,2010,1.0,john-wall,kentucky,kentucky,fr.,1.0,pg,kentucky,fr.,76.00,101.5,81.25,30.0,39.0
1,2010,1.0,evan-turner,ohio-state,ohio-state,jr.,2.0,sg,ohio-state,jr.,79.00,103.5,80.00,27.5,34.5
2,2010,1.0,derrick-favors,georgia-tech,georgia-tech,fr.,3.0,pf,georgia-tech,fr.,82.25,110.0,88.00,31.5,35.5
3,2010,1.0,wesley-johnson,syracuse,syracuse,jr.,4.0,sf,syracuse,jr.,79.25,106.0,85.00,32.0,37.0
4,2010,1.0,demarcus-cousins,kentucky,kentucky,fr.,7.0,pf/c,kentucky,fr.,82.75,113.0,89.75,23.5,27.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,2019,NaN,donta-hall,NaN,NaN,NaN,93.0,c,alabama,sr.,NaN,NaN,NaN,NaN,NaN
987,2019,NaN,bennie-boatwright,NaN,NaN,NaN,95.0,pf,southern-california,sr.,NaN,NaN,NaN,NaN,NaN
988,2019,NaN,cj-massinburg,NaN,NaN,NaN,96.0,sg,buffalo,sr.,NaN,NaN,NaN,NaN,NaN
989,2019,NaN,jarrey-foster,NaN,NaN,NaN,98.0,sf,smu,sr.,NaN,NaN,NaN,NaN,NaN


In [216]:
df_all.college_id_x = df_all.college_id_x.fillna(df_all.college_id_y)
df_all.college_id_x = df_all.college_id_x.fillna(df_all.college_id)
df_all.class_x = df_all.class_x.fillna(df_all.class_y)
df_all['round'] = df_all['round'].fillna(0)
df_all['bb_rank'] = df_all['bb_rank'].fillna(101)
df_all = df_all.drop(columns=['college_id_y','college_id','class_y'])
#df_all = df_all.dropna()

In [220]:
df_all

,draft_year,round,player_id,college_id_x,class_x,bb_rank,pos,ht_shoes,std_reach,wingspan,std_vert_leap,max_vert_leap
0,2010,1.0,john-wall,kentucky,fr.,1.0,pg,76.00,101.5,81.25,30.0,39.0
1,2010,1.0,evan-turner,ohio-state,jr.,2.0,sg,79.00,103.5,80.00,27.5,34.5
2,2010,1.0,derrick-favors,georgia-tech,fr.,3.0,pf,82.25,110.0,88.00,31.5,35.5
3,2010,1.0,wesley-johnson,syracuse,jr.,4.0,sf,79.25,106.0,85.00,32.0,37.0
4,2010,1.0,demarcus-cousins,kentucky,fr.,7.0,pf/c,82.75,113.0,89.75,23.5,27.5
...,...,...,...,...,...,...,...,...,...,...,...,...
986,2019,0.0,donta-hall,alabama,sr.,93.0,c,NaN,NaN,NaN,NaN,NaN
987,2019,0.0,bennie-boatwright,southern-california,sr.,95.0,pf,NaN,NaN,NaN,NaN,NaN
988,2019,0.0,cj-massinburg,buffalo,sr.,96.0,sg,NaN,NaN,NaN,NaN,NaN
989,2019,0.0,jarrey-foster,smu,sr.,98.0,sf,NaN,NaN,NaN,NaN,NaN


In [221]:
#Final csv file of nba draft prospects including their draft status and NBA combine stats
df_all.to_csv('data/prospects_combined.csv')